In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q flwr["simulation"] tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 

In [3]:
import flwr as fl
import tensorflow as tf
import numpy as np
from typing import List, Tuple
from flwr.common import Metrics
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
train_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_train.npy")
train_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_train.npy")
test_data = np.load("/content/drive/MyDrive/federated_learning_ecg/X_test.npy")
test_label = np.load("/content/drive/MyDrive/federated_learning_ecg/y_test.npy")

In [5]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(6):
  x_train.append(train_data[train_data.shape[0]//6*i:train_data.shape[0]//6*(i+1)])
  y_train.append(train_label[train_label.shape[0]//6*i:train_label.shape[0]//6*(i+1)])
  x_test.append(test_data[test_data.shape[0]//6*i:test_data.shape[0]//6*(i+1)])
  y_test.append(test_label[test_label.shape[0]//6*i:test_label.shape[0]//6*(i+1)])

In [6]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train_cid, y_train_cid, x_test_cid, y_test_cid) -> None:
        self.model = model
        self.x_train_cid, self.y_train_cid = x_train_cid, y_train_cid
        self.x_test_cid, self.y_test_cid = x_test_cid, y_test_cid

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train_cid, self.y_train_cid, epochs=5, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.x_train_cid), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, acc = self.model.evaluate(self.x_test_cid, self.y_test_cid, verbose=0)
        return loss, len(self.x_test_cid), {"accuracy": acc}

In [7]:
def client_fn(cid: str) -> fl.client.Client:

    model = Sequential([
      layers.Conv1D(16, 3, padding='same', activation='relu', input_shape=(180,1)),
      layers.MaxPool1D(),
      layers.Conv1D(32, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Conv1D(64, 3, padding='same', activation='relu'),
      layers.MaxPool1D(),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(5 ,activation="softmax")
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy()])

    x_train_cid = x_train[int(cid)]
    y_train_cid = y_train[int(cid)]

    x_test_cid = x_test[int(cid)]
    y_test_cid = y_test[int(cid)]


    return FlowerClient(model, x_train_cid, y_train_cid, x_test_cid, y_test_cid)

In [8]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:

    print(metrics)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]


    return {"accuracy": sum(accuracies) / sum(examples)}

In [9]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=6,
    min_evaluate_clients=3,
    min_available_clients=6,
    evaluate_metrics_aggregation_fn=weighted_average,
)

client_resources = None


fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=6,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-15 12:07:23,743 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-15 12:07:30,720	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-15 12:07:34,657 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3912636825.0, 'memory': 7825273652.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3912636825.0, 'memory': 7825273652.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-15 12:07:34,670 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-15 12:07:34,677 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=983) 2023-06-15 12:07:39.278125: W tensorflow/compiler/

[(5473, {'accuracy': 0.9143065810203552}), (5473, {'accuracy': 0.9144893288612366}), (5473, {'accuracy': 0.91923987865448})]


DEBUG flwr 2023-06-15 12:14:23,484 | server.py:232 | fit_round 2 received 6 results and 0 failures
DEBUG:flwr:fit_round 2 received 6 results and 0 failures
DEBUG flwr 2023-06-15 12:14:23,519 | server.py:168 | evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 2: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 12:14:26,531 | server.py:182 | evaluate_round 2 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 2 received 3 results and 0 failures
DEBUG flwr 2023-06-15 12:14:26,538 | server.py:218 | fit_round 3: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 3: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.9713137149810791}), (5473, {'accuracy': 0.9720445871353149}), (5473, {'accuracy': 0.9727754592895508})]


DEBUG flwr 2023-06-15 12:17:41,292 | server.py:232 | fit_round 3 received 6 results and 0 failures
DEBUG:flwr:fit_round 3 received 6 results and 0 failures
DEBUG flwr 2023-06-15 12:17:41,333 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 12:17:45,512 | server.py:182 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
DEBUG flwr 2023-06-15 12:17:45,519 | server.py:218 | fit_round 4: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 4: strategy sampled 6 clients (out of 6)


...

DEBUG flwr 2023-06-15 12:37:30,262 | server.py:232 | fit_round 9 received 6 results and 0 failures
DEBUG:flwr:fit_round 9 received 6 results and 0 failures
DEBUG flwr 2023-06-15 12:37:30,296 | server.py:168 | evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 9: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 12:37:33,787 | server.py:182 | evaluate_round 9 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 9 received 3 results and 0 failures
DEBUG flwr 2023-06-15 12:37:33,791 | server.py:218 | fit_round 10: strategy sampled 6 clients (out of 6)
DEBUG:flwr:fit_round 10: strategy sampled 6 clients (out of 6)


[(5473, {'accuracy': 0.9861136674880981}), (5473, {'accuracy': 0.9820939302444458}), (5473, {'accuracy': 0.9848346710205078})]


DEBUG flwr 2023-06-15 12:40:52,076 | server.py:232 | fit_round 10 received 6 results and 0 failures
DEBUG:flwr:fit_round 10 received 6 results and 0 failures
DEBUG flwr 2023-06-15 12:40:52,118 | server.py:168 | evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG:flwr:evaluate_round 10: strategy sampled 3 clients (out of 6)
DEBUG flwr 2023-06-15 12:40:55,902 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-06-15 12:40:55,907 | server.py:147 | FL finished in 1994.3750206409998
INFO:flwr:FL finished in 1994.3750206409998
INFO flwr 2023-06-15 12:40:55,914 | app.py:218 | app_fit: losses_distributed [(1, 0.2822443942228953), (2, 0.10885692636171977), (3, 0.08261571079492569), (4, 0.07671370108922322), (5, 0.0796190748612086), (6, 0.07485527296861012), (7, 0.0779732217391332), (8, 0.07814488063255946), (9, 0.08146481215953827), (10, 0.07982512315114339)]
INFO:flwr:app_fit: losses_dist

[(5473, {'accuracy': 0.9853827953338623}), (5473, {'accuracy': 0.9842864871025085}), (5473, {'accuracy': 0.9864791035652161})]


History (loss, distributed):
	round 1: 0.2822443942228953
	round 2: 0.10885692636171977
	round 3: 0.08261571079492569
	round 4: 0.07671370108922322
	round 5: 0.0796190748612086
	round 6: 0.07485527296861012
	round 7: 0.0779732217391332
	round 8: 0.07814488063255946
	round 9: 0.08146481215953827
	round 10: 0.07982512315114339
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.9160119295120239), (2, 0.9720445871353149), (3, 0.9791704813639323), (4, 0.9808758099873861), (5, 0.9834947387377421), (6, 0.9836774667104086), (7, 0.9834947188695272), (8, 0.9837383429209391), (9, 0.984347422917684), (10, 0.9853827953338623)]}